In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from pathlib import Path 
from unidip import UniDip
import unidip.dip as dip
from sorted_alpha import sorted_alphanumeric

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
path_boxes = '/Users/andreibirladeanu/Documents/Data/experimenting/visualize/1117_meal/skel1_tr1/'
boxes = [x for x in sorted_alphanumeric(os.listdir(path_boxes)) if x[0]!="."]
boxes = [cv2.imread(path_boxes + x) for x in boxes]

In [ ]:
boxes = [np.mean(box,dtype=np.float32, axis=2) for box in boxes]

In [ ]:
plt.imshow(np.mean(boxes[0],dtype=np.float16, axis=2))

In [ ]:
type(boxes[0])

In [ ]:
boxes[0].nbytes

In [ ]:
np.mean(boxes[0], axis=2).nbytes, np.mean(boxes[0], axis=2).shape

In [ ]:
new = np.round(np.mean(boxes[0], axis=2),2)

In [ ]:
np.round(np.mean(boxes[0], axis=2),2).nbytes

In [ ]:
np.mean(boxes[0],dtype=np.float32, axis=2).nbytes

In [ ]:
np.mean(boxes[0],dtype=np.float16, axis=2)

In [ ]:
np.mean(boxes[0], axis=2)

In [ ]:
def find_min_round(array):
    array = np.array(array)
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array)
    x = round(np.max(array))
    return(x)

In [ ]:
### mean across the first channel of all the pics
np.mean(boxes[0], axis=2).shape

In [ ]:
plt.imshow(np.mean(boxes[1], axis=2))


In [ ]:
for_hist =  np.array([[np.mean(x)] for x in boxes])
for_dip =  np.array([np.mean(x) for x in boxes])

In [ ]:
for_dip

In [ ]:
a  = np.zeros((2,3,2))
a.shape

In [ ]:
a

In [ ]:
np.mean(a, axis=2).shape[1]

In [ ]:
for x,y in zip(range(len(for_hist)), range(len(boxes))):
    if for_hist[x]<np.mean(for_hist):
        cv2.imshow('below mode',boxes[y])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=2, random_state=0).fit(for_hist)

In [ ]:
gm.means_

In [ ]:
labels = gm.predict(for_hist) 

In [ ]:
labels[0]

In [ ]:
np.where(labels==1)

In [ ]:
for x in np.where(labels==2)[0]:
    cv2.imshow('child', boxes[x])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
#### idea: get 3 gaussians and if the third one is less than 15% of the data throw it away 

In [ ]:
data = np.msort(for_dip)
print(dip.diptst(data))


In [ ]:
intervals

In [ ]:
unimodal = dip.diptst(data)

In [ ]:
unimodal[1]

In [ ]:
####### try it with bounding boxes

In [ ]:
path_openpose = '/Users/andreibirladeanu/Documents/Data/experimenting/tracked/1122_meal/skel1/'
coords = [file for file in sorted_alphanumeric(os.listdir(path_openpose)) if file[0] !="."]


In [ ]:
### get bounding boxes now 
path_video = '/Users/andreibirladeanu/Documents/Data/experimenting/videos/1122_meal.mp4'
path_boxes = '/Users/andreibirladeanu/Documents/Data/experimenting/bounding_boxes/skel1/'
cap = cv2.VideoCapture(path_video)

#for skel in skeletons1:
#    x_array = list(skel.values())[0][0]
#    y_array = list(skel.values())[0][1]
#    cap.set(1, list(skel.keys())[0])
#    ret, image = cap.read()
#    print(skel)
#    cv2.imwrite(path_boxes + str(list(skel.keys())[0]) + '.png', image[find_min_round(y_array):find_max_round(y_array),
#                   find_min_round(x_array):find_max_round(x_array)])

In [ ]:
for frame in coords:
    skel = pd.read_csv(path_openpose + frame)
    cap.set(1,int(frame.split('.')[0]))
    ret, image = cap.read()
    cv2.imwrite(path_boxes + str(frame.split('.')[0]) + '.png', image[find_min_round(skel['y']):find_max_round(skel['y']),
           find_min_round(skel['x']):find_max_round(skel['x'])])

In [ ]:
coords

In [ ]:
path = "/Users/andreibirladeanu/Documents/Data/experimenting/tracked/1117_meal/skel1_tr1/2.csv"
data = pd.read_csv(path)

In [ ]:
data['x'].loc[8]

In [ ]:
### lets try to draw proper contours around people
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if neck ==0:
        return(0)
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = np.subtract(hip1, neck)
        else:
            torso = np.subtract(hip2, neck)
    elif hip1[0]==0:
        torso = np.subtract(hip2, neck)
    else:
        torso = np.subtract(hip1, neck)
    return(torso)
        
        

In [ ]:
tuple1 = [2,3]
tuple2 = [1,2]

In [ ]:
np.subtract(tuple1, tuple2)

In [ ]:
torso_size(data)

In [ ]:
data